In [ ]:
# 05d — Poincaré embeddings for CIP hierarchy (IPEDs)

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from google.colab import drive

# Mount Drive
drive.mount('/content/drive', force_remount=True)

PROJ = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
OUT_DIR = f"{PROJ}/outputs/embeddings"
os.makedirs(OUT_DIR, exist_ok=True)

ipeds_path = f"{DATA_DIR}/ipeds_data.csv"
ipeds = pd.read_csv(ipeds_path)

print("Loaded IPEDS data:", ipeds.shape)
ipeds[["unitid", "year", "cips"]].head()


Mounted at /content/drive
Loaded IPEDS data: (11004, 21)


,unitid,year,cips
0,100654,2017,"1.0999,1.1001,1.9999,10.0202,11.0101,13.0101,1..."
1,100654,2018,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019..."
2,100654,2019,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019..."
3,100654,2020,"01.0999,01.1001,01.9999,03.0599,04.0301,05.029..."
4,100654,2021,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019..."


In [ ]:
# Parse comma-separated CIP strings to lists
ipeds["cips_list"] = (
    ipeds["cips"]
    .astype(str)
    .str.replace(" ", "")
    .str.split(",")
)

def normalize_cip(code: str):
    code = str(code).strip()
    if code == "" or code.lower() == "nan":
        return None
    if "." not in code:
        # keep things like "99" as-is
        return code
    left, right = code.split(".", 1)
    left = left.zfill(2)              # "1" -> "01"
    right = right.ljust(4, "0")[:4]   # pad to 4, then cut to length 4
    return f"{left}.{right}"

def normalize_cip_list(cip_list):
    out = []
    for c in cip_list:
        norm = normalize_cip(c)
        if norm is not None:
            out.append(norm)
    return out

ipeds["cips_list_norm"] = ipeds["cips_list"].apply(normalize_cip_list)

print("Example normalized CIP lists:")
ipeds[["unitid", "year", "cips", "cips_list_norm"]].head()


Example normalized CIP lists:


,unitid,year,cips,cips_list_norm
0,100654,2017,"1.0999,1.1001,1.9999,10.0202,11.0101,13.0101,1...","[01.0999, 01.1001, 01.9999, 10.0202, 11.0101, ..."
1,100654,2018,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019...","[01.0999, 01.1001, 01.9999, 03.0599, 04.0301, ..."
2,100654,2019,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019...","[01.0999, 01.1001, 01.9999, 03.0599, 04.0301, ..."
3,100654,2020,"01.0999,01.1001,01.9999,03.0599,04.0301,05.029...","[01.0999, 01.1001, 01.9999, 03.0599, 04.0301, ..."
4,100654,2021,"01.0999,01.1001,01.9999,03.0599,04.0301,09.019...","[01.0999, 01.1001, 01.9999, 03.0599, 04.0301, ..."


In [ ]:
# Collect all unique normalized CIP codes
all_cips = sorted({c for lst in ipeds["cips_list_norm"] for c in lst})
print("Unique CIP codes in IPEDS slice:", len(all_cips))
all_cips[:10]


Unique CIP codes in IPEDS slice: 1585


['01.0000',
 '01.0101',
 '01.0102',
 '01.0103',
 '01.0104',
 '01.0105',
 '01.0106',
 '01.0199',
 '01.0201',
 '01.0205']

In [ ]:
# Split a normalized CIP into hierarchy levels: L2, L4, L6
def split_levels(code: str):
    """
    Returns dict with levels:
      l2: 2-digit group (before dot, or entire code if no dot)
      l4: NN.MM (2-digit subgroup) if applicable
      l6: NN.MMMM (full 6-digit CIP) if applicable
    """
    if "." not in code:
        # treat codes like "99" as L2-only
        return {"l2": code, "l4": None, "l6": None}

    left, right = code.split(".")
    left = left  # already padded
    # right has 4 digits like "0901"
    l2 = left
    l4 = f"{left}.{right[:2]}" if len(right) >= 2 else None
    l6 = f"{left}.{right}" if len(right) >= 4 else None
    return {"l2": l2, "l4": l4, "l6": l6}

# Build node set & parent-child edges (L2->L4, L4->L6)
nodes = set()
edges = set()

# Also track L6 -> (L2, L4) for multi-hop positives
l6_to_l2 = {}
l6_to_l4 = {}

for code in all_cips:
    lv = split_levels(code)
    l2, l4, l6 = lv["l2"], lv["l4"], lv["l6"]

    # Always include L2
    nodes.add(l2)

    if l4 is not None:
        nodes.add(l4)
        edges.add((l2, l4))

    if l6 is not None:
        nodes.add(l6)
        if l4 is not None:
            edges.add((l4, l6))
        else:
            edges.add((l2, l6))

        l6_to_l2[l6] = l2
        l6_to_l4[l6] = l4

nodes = sorted(nodes)
print("Total hierarchy nodes:", len(nodes))
print("Total parent-child edges:", len(edges))
list(edges)[:10]


Total hierarchy nodes: 2018
Total parent-child edges: 1978


[('13.13', '13.1302'),
 ('45.99', '45.9999'),
 ('04.08', '04.0802'),
 ('01.10', '01.1005'),
 ('11.01', '11.0199'),
 ('16.08', '16.0801'),
 ('51.33', '51.3301'),
 ('26.03', '26.0305'),
 ('15.13', '15.1303'),
 ('43.01', '43.0107')]

In [ ]:
# Build groups of L6 by L4 and L2
from collections import defaultdict
import itertools

l4_groups = defaultdict(list)
l2_groups = defaultdict(list)

for l6, l2 in l6_to_l2.items():
    l2_groups[l2].append(l6)
    l4 = l6_to_l4[l6]
    if l4 is not None:
        l4_groups[l4].append(l6)

# Multi-hop positives: all pairs of L6 codes sharing L4 or L2
multi_pos_pairs = set()

# Same L4 (tighter)
for l4, members in l4_groups.items():
    if len(members) < 2:
        continue
    for a, b in itertools.combinations(members, 2):
        multi_pos_pairs.add((a, b))

# Same L2 (broader)
for l2, members in l2_groups.items():
    if len(members) < 2:
        continue
    # you can subsample if you want; here we do all
    for a, b in itertools.combinations(members, 2):
        multi_pos_pairs.add((a, b))

print("Multi-hop positive pairs:", len(multi_pos_pairs))
list(multi_pos_pairs)[:10]


Multi-hop positive pairs: 67996


[('45.0602', '45.0605'),
 ('51.1108', '51.1299'),
 ('51.0699', '51.1107'),
 ('50.0406', '50.1099'),
 ('51.2007', '51.3999'),
 ('51.0799', '51.1507'),
 ('13.0499', '13.1309'),
 ('51.0723', '51.3824'),
 ('51.1701', '51.2799'),
 ('19.0202', '19.0901')]

In [ ]:
# Combine direct hierarchy edges + multi-hop positives
pos_pairs_str = set()

# add both directions for symmetry
for u, v in edges:
    pos_pairs_str.add((u, v))
    pos_pairs_str.add((v, u))

for u, v in multi_pos_pairs:
    pos_pairs_str.add((u, v))
    pos_pairs_str.add((v, u))

print("Total positive pairs (with symmetry):", len(pos_pairs_str))
list(pos_pairs_str)[:10]


Total positive pairs (with symmetry): 139948


[('45.0602', '45.0605'),
 ('51.2007', '51.3999'),
 ('13.1335', '13.1208'),
 ('50.0406', '50.1099'),
 ('51.0799', '51.1507'),
 ('13.0499', '13.1309'),
 ('52.1601', '52.1499'),
 ('19.0202', '19.0901'),
 ('51.2207', '51.3808'),
 ('51.1803', '51.1502')]

In [ ]:
# Map node -> depth (0=L2, 1=L4, 2=L6)
def node_depth(node: str):
    if "." not in node:
        return 0  # L2 or weird code like "99"
    left, right = node.split(".")
    if len(right) == 2:
        return 1  # L4
    elif len(right) == 4:
        return 2  # L6
    # fallback
    return 2

node_to_idx = {n: i for i, n in enumerate(nodes)}
idx_to_node = {i: n for n, i in node_to_idx.items()}
num_nodes = len(nodes)

depths = np.array([node_depth(n) for n in nodes], dtype=np.int64)
print("Depth counts:", {d: (depths == d).sum() for d in np.unique(depths)})


Depth counts: {np.int64(0): np.int64(40), np.int64(1): np.int64(404), np.int64(2): np.int64(1574)}


In [ ]:
import torch

pos_pairs_idx = torch.tensor(
    [[node_to_idx[u], node_to_idx[v]] for (u, v) in pos_pairs_str],
    dtype=torch.long
)
print("pos_pairs_idx shape:", pos_pairs_idx.shape)
pos_pairs_idx[:5]


pos_pairs_idx shape: torch.Size([139948, 2])


tensor([[1367, 1370],
        [1758, 1881],
        [ 447,  405],
        [1529, 1591],
        [1651, 1734]])

In [ ]:
from collections import defaultdict

neighbors = defaultdict(set)
for u, v in pos_pairs_str:
    neighbors[u].add(v)
    neighbors[v].add(u)  # symmetric

# Convert to index-based dictionary for speed
neighbors_idx = {node_to_idx[u]: {node_to_idx[v] for v in vs} for u, vs in neighbors.items()}


In [ ]:
class PoincareEmb(nn.Module):
    def __init__(self, num_nodes, dim, init_scale=1e-3, max_radius=0.9):
        super().__init__()
        self.raw = nn.Parameter(torch.randn(num_nodes, dim) * init_scale)
        self.max_radius = max_radius

    def embeddings(self):
        # project to ball via tanh and radius scaling
        x = torch.tanh(self.raw)
        # scale each vector to <= max_radius
        norms = torch.norm(x, dim=1, keepdim=True) + 1e-9
        scale = torch.clamp(self.max_radius / norms, max=1.0)
        return x * scale

def poincare_dist(x, y, eps=1e-5):
    """
    x, y: (..., dim) points in the open unit ball
    Returns hyperbolic distance d(x,y).
    """
    x2 = (x * x).sum(dim=-1, keepdim=True)
    y2 = (y * y).sum(dim=-1, keepdim=True)
    diff2 = ((x - y) ** 2).sum(dim=-1, keepdim=True)

    num = 2.0 * diff2
    denom = (1 - x2).clamp(min=eps) * (1 - y2).clamp(min=eps)
    z = 1.0 + num / denom
    z = z.clamp(min=1.0 + eps)
    return torch.acosh(z).squeeze(-1)

# Depth-based target radii (can tweak)
R_TARGET = {
    0: 0.25,  # L2
    1: 0.55,  # L4
    2: 0.80,  # L6
}

def radius_regularizer(emb, depths_t, lam=1.0):
    """
    Penalize deviation of ||x|| from target radius per depth.
    emb: (num_nodes, dim)
    depths_t: (num_nodes,)
    """
    norms = torch.norm(emb, dim=1)  # (num_nodes,)
    target = torch.zeros_like(norms)
    for d, r in R_TARGET.items():
        mask = (depths_t == d)
        target[mask] = r
    return lam * ((norms - target) ** 2).mean()


In [ ]:
def poincare_loss(
    model,
    pos_pairs_idx,
    neighbors_idx,
    num_nodes,
    neg_samples=10,
    batch_size=1024,
    device="cpu",
    reg_weight=1.0,
    depths_np=None,
):
    model.train()
    emb = model.embeddings()  # (N, dim)

    # Sample a minibatch of positive pairs
    num_pos = pos_pairs_idx.shape[0]
    batch_idx = torch.randint(0, num_pos, (batch_size,), device=device)
    batch_pairs = pos_pairs_idx[batch_idx]

    u_idx = batch_pairs[:, 0]
    v_idx = batch_pairs[:, 1]

    u = emb[u_idx]
    v = emb[v_idx]

    d_pos = poincare_dist(u, v)  # (batch_size,)

    # Negative sampling: pick random nodes, avoid true neighbors when possible
    neg_v_idx = torch.randint(0, num_nodes, (batch_size, neg_samples), device=device)
    # (optional / light filtering)
    # you could mask neighbors here but it's already a lot of pairs

    neg_v = emb[neg_v_idx]  # (batch_size, neg_samples, dim)
    u_expanded = u.unsqueeze(1).expand_as(neg_v)

    d_neg = poincare_dist(u_expanded, neg_v)  # (batch_size, neg_samples)

    # Softplus ranking loss: encourage d_pos < d_neg
    # loss = E[softplus(d_pos - d_neg)]
    loss_rank = torch.nn.functional.softplus(
        d_pos.unsqueeze(1) - d_neg
    ).mean()

    # Radius regularizer
    depths_t = torch.tensor(depths_np, device=device, dtype=torch.long)
    loss_reg = radius_regularizer(emb, depths_t, lam=reg_weight)

    return loss_rank + loss_reg, loss_rank.item(), loss_reg.item()


In [ ]:
dim = 64
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model_64 = PoincareEmb(num_nodes=num_nodes, dim=dim).to(device)
pos_pairs_idx_t = pos_pairs_idx.to(device)

optimizer = optim.Adam(model_64.parameters(), lr=0.01)

epochs = 50
batch_size = 2048
neg_samples = 10
reg_weight = 1.0

for epoch in range(1, epochs + 1):
    loss, loss_rank_val, loss_reg_val = poincare_loss(
        model_64,
        pos_pairs_idx_t,
        neighbors_idx,
        num_nodes=num_nodes,
        neg_samples=neg_samples,
        batch_size=batch_size,
        device=device,
        reg_weight=reg_weight,
        depths_np=depths,
    )
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch == 1 or epoch % 5 == 0:
        print(
            f"Epoch {epoch:02d}/{epochs} "
            f"total={loss.item():.4f} "
            f"rank={loss_rank_val:.4f} "
            f"reg={loss_reg_val:.4f}"
        )


Using device: cpu
Epoch 01/50 total=1.2425 rank=0.6931 reg=0.5493
Epoch 05/50 total=0.8789 rank=0.6385 reg=0.2403
Epoch 10/50 total=0.5868 rank=0.5008 reg=0.0860
Epoch 15/50 total=0.4758 rank=0.4183 reg=0.0575
Epoch 20/50 total=0.4095 rank=0.3706 reg=0.0390
Epoch 25/50 total=0.3491 rank=0.3126 reg=0.0365
Epoch 30/50 total=0.3136 rank=0.2760 reg=0.0375
Epoch 35/50 total=0.2890 rank=0.2515 reg=0.0375
Epoch 40/50 total=0.2634 rank=0.2260 reg=0.0374
Epoch 45/50 total=0.2364 rank=0.1990 reg=0.0374
Epoch 50/50 total=0.2235 rank=0.1860 reg=0.0374


In [ ]:
# ---- Extract final embeddings from the trained model ----
model_64.eval()
with torch.no_grad():
    emb_64 = model_64.embeddings().cpu().numpy()  # (num_nodes, 64)

poinc_cols = [f"poinc_{i}" for i in range(64)]
emb_df = pd.DataFrame(emb_64, columns=poinc_cols)
emb_df.insert(0, "node", nodes)

print("Full hierarchy embedding shape:", emb_df.shape)
display(emb_df.head())


Full hierarchy embedding shape: (2018, 65)


,node,poinc_0,poinc_1,poinc_2,poinc_3,poinc_4,poinc_5,poinc_6,poinc_7,poinc_8,...,poinc_54,poinc_55,poinc_56,poinc_57,poinc_58,poinc_59,poinc_60,poinc_61,poinc_62,poinc_63
0,01,-0.055052,0.033951,-0.022616,0.098721,0.069493,0.102743,0.068059,0.116952,-0.057324,...,-0.066338,0.019994,0.099413,0.003386,0.049719,-0.027094,0.113460,0.054250,0.054832,-0.098273
1,01.00,-0.133126,-0.077652,0.092110,-0.089352,-0.105932,0.139320,-0.098445,0.121089,-0.132556,...,0.098774,-0.077032,0.117177,-0.100882,0.140650,0.099422,-0.112889,0.138622,-0.083527,-0.127347
2,01.0000,0.090471,0.197550,0.043015,-0.056116,-0.061400,-0.038373,0.075201,0.093177,0.168469,...,0.187417,0.125314,0.042074,-0.136623,-0.036190,0.069549,-0.049810,-0.101078,0.209098,-0.066828
3,01.01,-0.103971,0.141359,-0.088766,0.107218,0.113957,0.092299,0.120782,0.132587,-0.092987,...,0.110773,-0.080102,0.132561,-0.126698,0.118158,-0.112997,0.104309,0.104025,0.143806,-0.120890
4,01.0101,0.147366,0.201431,0.071400,-0.046829,-0.017985,-0.110639,0.071759,0.137153,0.179083,...,0.141914,0.106939,0.111453,-0.115059,-0.028334,0.077553,0.050344,-0.139364,0.199735,-0.055682


In [ ]:
def is_l6(node: str):
    if "." not in node:
        return False
    left, right = node.split(".")
    return len(right) == 4

cip_poinc_64_df = emb_df[emb_df["node"].apply(is_l6)].copy()
cip_poinc_64_df.rename(columns={"node": "cip"}, inplace=True)

print("L6 CIP Poincaré 64D shape:", cip_poinc_64_df.shape)
display(cip_poinc_64_df.head())


L6 CIP Poincaré 64D shape: (1574, 65)


,cip,poinc_0,poinc_1,poinc_2,poinc_3,poinc_4,poinc_5,poinc_6,poinc_7,poinc_8,...,poinc_54,poinc_55,poinc_56,poinc_57,poinc_58,poinc_59,poinc_60,poinc_61,poinc_62,poinc_63
2,01.0000,0.090471,0.197550,0.043015,-0.056116,-0.061400,-0.038373,0.075201,0.093177,0.168469,...,0.187417,0.125314,0.042074,-0.136623,-0.036190,0.069549,-0.049810,-0.101078,0.209098,-0.066828
4,01.0101,0.147366,0.201431,0.071400,-0.046829,-0.017985,-0.110639,0.071759,0.137153,0.179083,...,0.141914,0.106939,0.111453,-0.115059,-0.028334,0.077553,0.050344,-0.139364,0.199735,-0.055682
5,01.0102,0.133944,0.199155,-0.000136,-0.014325,0.008370,-0.040488,0.082161,0.140083,0.152745,...,0.143322,0.130878,0.138846,-0.104361,0.011743,0.016922,-0.028874,-0.106440,0.204865,-0.071772
6,01.0103,0.051900,0.203464,0.013503,0.000496,-0.079537,-0.097635,0.053124,0.120365,0.162752,...,0.155462,0.114426,0.127033,-0.117831,0.025497,0.046080,-0.035165,-0.079472,0.187237,-0.050682
7,01.0104,0.069693,0.196363,0.075168,-0.048550,-0.027041,-0.113763,0.057446,0.139221,0.204076,...,0.096767,0.104519,0.160370,-0.114698,-0.006529,0.062369,-0.042677,-0.057034,0.212512,0.003987


In [ ]:
PROJ = "/content/drive/MyDrive/dissertation"
OUT_DIR = f"{PROJ}/outputs/embeddings"
os.makedirs(OUT_DIR, exist_ok=True)

local_path_64 = "/content/cip_embeddings_poincare_ipeds_64.csv"
drive_path_64 = f"{OUT_DIR}/cip_embeddings_poincare_ipeds_64.csv"

cip_poinc_64_df.to_csv(local_path_64, index=False)
cip_poinc_64_df.to_csv(drive_path_64, index=False)

print("Saved 64D Poincaré embeddings to:")
print(" -", local_path_64)
print(" -", drive_path_64)


Saved 64D Poincaré embeddings to:
 - /content/cip_embeddings_poincare_ipeds_64.csv
 - /content/drive/MyDrive/dissertation/outputs/embeddings/cip_embeddings_poincare_ipeds_64.csv
